In [ ]:
#!pip3 install fastai==1.0.42

In [ ]:
import fastai

In [ ]:
fastai.__version__

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from fastai import *
from fastai.text import * 
from fastai.gen_doc.nbdoc import *
from fastai.datasets import * 
from fastai.datasets import Config
from pathlib import Path
import pandas as pd

In [ ]:
path = Path("../input")

In [ ]:
!ls '../input/'

In [ ]:
!unzip '../input/*.zip' 

In [ ]:
train_df = pd.read_csv('/kaggle/working/train.csv')#.sample(frac=0.01)
#train_df['is_valid'] = 0
train_df.head(5)

In [ ]:
#train_df.loc[:50, 'is_valid'] = 1
#train_df.head()

In [ ]:
test_df = pd.read_csv('/kaggle/working/test.csv')#.sample(frac=0.01)
#test_df['is_valid'] = 0
test_df.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(train_df, test_size=0.2)

In [ ]:
#data_lm = TextLMDataBunch.from_df(path, 
#                                   train_df,
#                                   valid_df,
#                                   text_cols='comment_text')

In [ ]:
#??TextList.from_csv()

In [ ]:
data_lm = (TextList.from_csv(path, '/kaggle/working/train.csv', cols='comment_text') 
                   .random_split_by_pct(0.3)
                   .label_for_lm()
                   .add_test(TextList.from_csv(path, '/kaggle/working/test.csv', cols='comment_text'))
                   .databunch())

In [ ]:
data_lm

In [ ]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, model_dir="/tmp/model/", drop_mult=0.3)
learn.model

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))
#learn.fit_one_cycle(1, 1e-1)
learn.save('mini_train_lm')
learn.save_encoder('mini_train_encoder')

In [ ]:
#learn.unfreeze()
#learn.fit_one_cycle(1, 1e-3, moms=(0.8,0.7))

In [ ]:
learn.predict("Why the edits made under ", n_words=10)

In [ ]:
learn.show_results()

# Classifier

In [ ]:
data_clas = (TextList.from_csv(path, '/kaggle/working/train.csv',cols='comment_text', vocab=data_lm.vocab) #test='test'
    #.split_from_df(col='is_valid') #is_valid
    .random_split_by_pct(valid_pct=0.2)
    .label_from_df(cols=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])
    .add_test(TextList.from_csv(path, '/kaggle/working/test.csv', cols='comment_text'))
    .databunch(bs=42))

In [ ]:
# Classifier model data
#data_clas = TextClasDataBunch.from_csv(path, 'train.csv', vocab=data_lm.train_ds.vocab, bs=32)

In [ ]:
#from sklearn.model_selection import train_test_split

#train_df, valid_df = train_test_split(train_df, test_size=0.2)

In [ ]:
#??TextClasDataBunch.from_df()

In [ ]:
#data_clas = (TextClasDataBunch.from_df(path,train_df, valid_df, test_df, 
#                                   vocab=data_lm.vocab, 
#                                   text_cols='comment_text', 
#                                   label_cols=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])
             #.random_split_by_pct(valid_pct=0.2)
#             .split_from_df(col='is_valid') #is_valid
#             .databunch(bs=42) )

In [ ]:
#data_clas.show_batch()

In [ ]:
#data_clas

In [ ]:
data_clas.c

In [ ]:
learn = text_classifier_learner(data_clas, model_dir="/tmp/model/", drop_mult=0.5) #metrics=[accuracy_thresh]
learn.load_encoder('mini_train_encoder')
learn.freeze()
learn.model

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
#learn.crit = F.cross_entropy

In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)

In [ ]:
learn.metrics = metrics=[acc_02, f_score] #[top_k_accuracy]

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))
#learn.fit_one_cycle(1, slice(1e-3,1e-2))
learn.save('mini_train_clas')

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
#learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
#learn.freeze_to(-3)
#learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
#learn.unfreeze()
#learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

#learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

In [ ]:
learn.show_results()

In [ ]:
probs, _ = learn.get_preds(DatasetType.Test,ordered=True)

In [ ]:
#preds = probs.argmax(dim=1)

In [ ]:
#preds.shape

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
submission = pd.DataFrame({'id': test_df["id"]})

In [ ]:
submission = pd.concat([submission, pd.DataFrame(probs.numpy(), columns = label_cols)], axis=1)

In [ ]:
submission.to_csv('submission.csv', index=False)
submission.head(n=10)